In [22]:
import numpy as np
import pandas as pd
import pickle
import os
import jieba

# load data

In [18]:
with open("./data/data_input.pkl", "rb") as f:
    text_train, label_train, text_test, label_test, label_names = pickle.load(f)

# 分词

In [24]:
def text_to_tokens(text, stop_words = []):
    """
    text: 需要分词的文本
    stop_words: 停用词表，如果有的话
    """
    words = jieba.cut(text)
    tokens = [word for word in words if word not in stop_words]
    return tokens

In [25]:
train_tokens = np.array([text_to_tokens(text) for text in text_train])
test_tokens = np.array([text_to_tokens(text) for text in text_test])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/mengzhen/python3-tf-gpu/lib/python3.6/site-packages/jieba/__init__.py", line 152, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpbcpkk2qn' -> '/tmp/jieba.cache'
Loading model cost 2.641 seconds.
Prefix dict has been built succesfully.


In [27]:
print("train size: ", train_tokens.shape)
print("test size: ", test_tokens.shape)

train size:  (5734,)
test size:  (3656,)


# Tfidf 表征文本

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
corpus_train = [" ".join(tokens) for tokens in train_tokens]
corpus_test = [" ".join(tokens) for tokens in test_tokens]

In [38]:
tfidf_vector = TfidfVectorizer(max_features=5000)
tfidf_vector.fit(corpus_train + corpus_test)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [39]:
X_train = tfidf_vector.transform(corpus_train)
X_test = tfidf_vector.transform(corpus_test)

In [42]:
print("data train shape: ", X_train.shape)
print("data test shape: ", X_test.shape)

data train shape:  (5734, 5000)
data test shape:  (3656, 5000)


# 训练分类模型

In [23]:
import xgboost

In [48]:
xgb_cls = xgboost.XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=100, n_jobs=6)

In [ ]:
xgb_cls.fit(X_train, label_train)

# test

In [53]:
# predict labels or predict probability
test_pred = xgb_cls.predict(X_test)

# evaluation

In [2]:
from sklearn.metrics import classification_report
print(classification_report(y_pred= test_pred, y_true=label_test))